In [2]:
from bs4 import BeautifulSoup
import requests
import urllib
import json 
import unicodedata

In [ ]:
def get_onion_image_url(link):
    hdr = {'User-Agent': 'Mozilla/5.0'}  # Use firefox so no HTTP Error 403
    r = requests.get(link, headers=hdr)
    data = r.text  # Get the website source as text
    soup = BeautifulSoup(data) # Setup a "soup" which BeautifulSoup can search
    vals =  soup.findAll(attrs={"name":"twitter:image"})
    for v in vals:
#         make sure that the image is not the onion's logo
        if v["content"] not in ("https://x.kinja-static.com/assets/images/logos/placeholders/theonion.png", ):
            return v["content"]

In [ ]:
def get_hp_image_url(link):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(link, headers=hdr)
    data = r.text  #Get the website source as text
    soup = BeautifulSoup(data) #Setup a "soup" which BeautifulSoup can search
    vals = soup.findAll(attrs={"property":"og:image:url"})
    for v in vals:
#         make sure that the image is not the huffpost's logo
        if v["content"] not in ("https://img.huffingtonpost.com/asset/default-entry.jpg?ops=1778_1000"):
            return v["content"]

In [ ]:
# download article images
with open('Sarcasm_Headlines_Dataset_v2.json', 'r') as handle:
    json_data = [json.loads(line) for line in handle]
    for i, d in enumerate(json_data):
        try:
            link = d["article_link"]
            if d["is_sarcastic"] == 1:
                image_link = get_onion_image_url(link)
                urllib.request.urlretrieve(image_link, "satire_images/" + str(i) + ".jpg")
            else:
                image_link = get_hp_image_url(link)
                urllib.request.urlretrieve(image_link, "serious_images/" + str(i) + ".jpg")
            json_data[i]["has_image"] = True
        except:
            json_data[i]["has_image"] = False

with open("dataset_with_images_fulltext", "w") as outfile: 
    json.dump(json_data, outfile)
    

In [5]:
# download article body text
with open("dataset_with_images_fulltext.json", "r") as handle:
    json_data = json.load(handle)
    for i, d in enumerate(json_data):
        try:
            link = d["article_link"]
            r = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            data = r.text  
            soup = BeautifulSoup(data) 
            body_text = ""
            for content in soup.findAll('p'):  # Find content in paragraphs
                body_text += content.text + "\n"
            json_data[i]["text"] = unicodedata.normalize("NFKD", body_text)
        except:
            json_data[i]["text"] = ""
        
with open("dataset_with_images_fulltext.json", "w", encoding="utf-8") as outfile: 
    json.dump(json_data, outfile, ensure_ascii=False)
    